# Bank Heist Problem

## Description:
<br>
Given list of bank locations, how much money each one holds and the time it would take to rob each one, you need to make as much profit as possible! I bet you didn't think that data science could be this profitable! <br>

The list of banks is in `data/bank_data.csv`. This is what the data looks like.


`id, x_coordinate, y_coordinate, money, time (hr)
0, 11.4, 3.3, 5000, 0.6
1, 6.9, 7.1, 15000, 0.3
2, 1.4, 13.2, 900, 1.1`<br><br>
You have 24 hours to make as much money as possible and then escape.


## Rules:


<li>Your run can start anywhere on the map but it has to end at the helicopter escape zone: coordinates (0,0)</li>
<ul>
<li>If you try to rob too many banks and can't get to the helicopter in 24 hours, you get caught and go to jail.</li></ul>


<li>
Your solution is a list or array of integers (eg. [580, 433, 24, 998]) where the numbers are the IDs of each banks. The ID of each bank is their index (their row index).
</li>

<li>You travel between banks at 30 km/h. You have to travel from one bank to the next!</li>

<li>Remember the formula to calculate the distance between two points.</li>
<li>The coordinates are in kilometers.
So (1, 1) and (1, 2) are one kilometer apart.</li>
<li>This would take 1 / 30 hour = 2 minutes to travel
Your solution should be an approximative/heuristic algorithm</li>

<li>This problem is NP-Hard, you won't find a fast enough algorithm that has the perfect solution</li>
<li>It doesn't need to find the best solution</li>
<li>Find the best solution you can!</li>
<li>Your solution has to run on a common laptop in under 3 minutes for the 10,000 cities dataset</li>

<li>You can use everything you want to achieve this:</li>
<li>Use numpy, pandas, functions, algorithms</li>
<li>You can use parallelism</li>
<li>Use optimied libraries (pandas, numba, scipy, etc.)</li>
<li>Test your code on small subsets of the data so they run fast</li>
<li>Then scale your code up to bigger chunks of the data</li>
<li>Your input is a pandas dataframe with the bank data. Your output is a list of bank IDs in order that you rob them:</li>

### example
`df = pd.read_csv('bank_data.csv')
robber_algorithm(df)`

### Output is a list of bank IDs
[OUTPUT] --> [664, 2341, 26, 998, 9583, 24, 1, 444, 6783]<br>
Check Your Solution
You can use the `check_solution` function from `utils/check_solution.py` to test the validity of your solution and obtain a score.

In [95]:
import warnings
import pandas as pd 
import math
from pandas.core.common import SettingWithCopyWarning
from utils.check_solution import check_solution

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

### Importing Data


In [96]:
data = pd.read_csv('./data/bank_data.csv')

### We create a copy of this data ( just to be on the safe side )

In [97]:
datac = data.copy()

## Brainstorming:

<p>
We first must acknowledge this is a greedy algorithm.
    
We have multiple ways to approach this problem:
    <ul>
        <li>Traverse according to which bank has the most money.</li>
        <li>Traverse according to which bank takes the least amount of time to rob.</li>
        <li>Traverse according to which bank takes the least amount of time to reach and rob.</li>
        ... and many more
    </ul>
    
For this algorithm we will traverse the banks depending on which bank is closer to us regardless of how much time it takes to rob!
</p>

## We create the necessary helper functions to help us through this process.
### For more information please read the docstrings for each function provided below

In [110]:
# find time to arrive
def time_to_arrive(s,e):
    
    """
    This function calculates the distance between two points on a 2d Plane
    
    Parameters:
    
    s (tuple): coordinates of starting point
    e (tuple): coordinates of ending point
    
    returns:

    float: returning value
    
    
    """
    
    
    return math.sqrt((e[0]-s[0])**2 + (e[0]-s[1])**2)/30
    


# what is the total time to rob?
def total_time_to_rob(start, bank):
    
    """
    This function calculates the time to travel to the bank and the time to rob it
    
    Parameters:
    
    start (tuple): coordinates of starting point
    bank (series): row data of the bank we desire to rob
    
    returns:

    float: returning value
    
    
    """
    
    # we get the floating value for the x and y coordinates and store them in a temporary variable to access later
    
    ex = bank['x_coordinate']
    
    ey = bank['y_coordinate']
    
    end = (ex,ey)
    
    # we calculate the time to arrive from start to end and add the duration of the robbing time
    return time_to_arrive(start, end) + bank['time (hr)']

# remove the bank robbed from dataframe
def clear_bank(df, bank):
    """
    This function removes the bank we stole from the dataframe
    
    Parameters:
    
    df (DataFrame): Dataframe we are working on and want to search through
    bank (Series): bank row we want to remove
    
    returns:

    Dataframe: returning dataframe without that row
    
    
    """
    return df.drop(int(bank['id']), axis = 0,  inplace = True)


# find closest bank according to distance
def closest_bank(s, df):
    
    """
    This function calculates the closest bank from starting point s
    
    Parameters:
    
    s (tuple): coordinates of starting point
    df (DataFrame): Dataframe we are working on and want to search through
    
    returns:

    res(Series): Row of the bank closest to where we start from
    
    
    """
    # this is our duration time to reach our destinantion
    # it is just a large value to later find the minimum result in our dataframe
    # this would of course mean we don't have a bank that would require d's initial value to reach
    
    d = 9999999
    
    res = None
    
    # we iterate over the rows in the DataFrame
    
    for index, row in df.iterrows():
        
        # We store the rows x and y coordinates for later use
        
        ex = row['x_coordinate']
        
        ey = row['y_coordinate']
        
        end = (ex,ey)
        
        # we calculate the time needed to get from our starting location to the bank in the row examined
        
        t = time_to_arrive(s,row)
        
        # we check whether that time was less than the previous least time to get to a bank
        
        if (t < d):
            d = t
            res = row
            
    # we return the row that takes the least time to reach
    return res

# check if we have time to escape after the robbery attempt
def can_rob(hrs, start, bank):
    
    """
    This function calculates if we have time to rob that bank and make it in time to the escape point (0,0)
    
    Parameters:
    
    hrs (float): Hours left for us to escape
    start (tuple): coordinates of starting point
    bank (series): Bank we want to rob
    
    returns:

    Bool: Whether we can rob it or not
    
    
    """
    
    ex = bank['x_coordinate']
    
    ey = bank['y_coordinate']
    
    end = (ex,ey)

    # This piece of code is extremely cruicial to the logical flow of our algorithm
    # We check if we still have time to reach the escape point after we rob the next bank
    
    if (hrs - total_time_to_rob(start, bank)) > time_to_arrive(end, (0,0)) and time_to_arrive(end, (0,0)) > 0:
        return True
    
    # oops gotta go find another bank or escape NOW!
    return False


# find where is the bank with least time to rob
def get_bank_with_least_rob_time(test):
    return test[test['time (hr)'] == min(test['time (hr)'])]

### This is the heart of the code where the magic happens


In [112]:


def rob(df, hrs):
    
    """
    This function returns the list of banks we can rob in 24hrs and make it safely to the escape point
    
    Parameters:
    
    df (DataFrame): The DataFrame of banks we have to steal
    hrs (float): Time limit we have in hours
    
    returns:

    banks_robbed (List): List of Bank ids we can steal from
    
    
    """
    
    # since we are super safe thieves we wouldn't even want our copy of the original data to be tampered with,
    # and this function requires us to remove the bank row we robbed from the banks list
    # we create a temporary "draft" of the dataframe to safely remove elements from
    
    temp_df = df.copy()
    
    # starting position is bank that takes least time to rob
    
    sp = get_bank_with_least_rob_time(temp_df)
    
    start = (sp['x_coordinate'].values[0], sp['y_coordinate'].values[0])
    
    # we rob starting here

    banks_robbed = [sp['id'].values[0]]

    money = sp['money'].values[0]
    
    # rob ends
    
    # we remove the bank we robbed from our list
    
    clear_bank(temp_df, sp)
    
    # we search for the next closest bank
    
    cb = closest_bank(start, temp_df)
    
    # the time we have left
    
    time_left = hrs
    
    # This basically means that as long as we have banks to rob and we have the time to rob them... we shall continue robbing

    while ((not temp_df.empty) and can_rob(time_left, start, cb)):
        
#         print(f"cb: {cb}") 

        # cha-ching!

        money += cb['money']
        
        # We don't want to be the thieves who visit the crime scene ;)

        banks_robbed.append(int(cb['id']))
        
        # the time left for us after robbing the bank
        
        time_left = time_left - total_time_to_rob(start, cb)
        
        # this is our new starting location
        
        start = (cb['x_coordinate'], cb['y_coordinate'])
        
#         print(f"starting coordinates: {start}")

        # remove the bank we robbed from the list
        
        clear_bank(temp_df, cb)
        
        # search for new nearest bank
        
        cb = closest_bank(start, temp_df)
    
    return banks_robbed

robbed_banks = rob(datac,24)

### Let us check our score from the script provided for us

In [100]:
check_solution(robbed_banks, data)

Time Remaining: 1.1922581410229949


683700.0

# Conclusion:

<p>

While we ended up with a decent amount of cash. 
    
It is important to remember that there are multiple ways to plan these heists in a way that will be much more profitable.

We had a time remaining of 1.19 hrs which means our algorithm uses only 95% of our time available.
    
This of course means we can find a much better algorithm that uses the time we have more efficiently.
  
</p>



<i> Done by: Ahmad Naji -  </i>